# Interactively create traning data for building a model from multiple images

Create the traininig data needed for running InSegtPy from multiple training data


In [1]:
# Import the modules needed 
import insegtpy
import insegtpy.models
import skimage.io
import matplotlib.pyplot as plt
import PIL
import numpy as np
import os
import glob
import pickle
import time

%matplotlib notebook

In [2]:
# Interactive labeling

## Change to directory and image name of choice
#dir_in = '/Users/abda/Documents/Center/QIM/data/20220408_fibers/Group_0' 
dir_in = '/Users/vand/Documents/PROJECTS2/InSegtPy/fibers_christian/Group_0'

im_name = 'CFRP_00000.tiff'
##

image_train = skimage.io.imread(os.path.join(dir_in, im_name))

# Segmentation model
model = insegtpy.models.gauss_features_segmentor(image_train, 
                                   branching_factor = 5, 
                                   number_layers = 4,
                                   number_training_vectors = 4096,
                                   features_sigma = [1,2],
                                   propagation_size = 9, 
                                   scales=[1],
                                   propagation_repetitions=2)

# Start the annotation GUI
ex = insegtpy.insegt(image_train, model)

Bulding single-scale GaussFeatSegt model.
Number of images 1
Number of training patches 4096
Number of layers 4 number of elements 780
Adding propagation repetitions.
Starting InSegt Annotator. For help, hit 'H'.


In [3]:
#%% Create the labeled data to train from multiple images

data_dirs = []
data_dirs.append('/Users/abda/Documents/Center/QIM/data/20220408_fibers/Group_0/')
data_dirs.append('/Users/abda/Documents/Center/QIM/data/20220408_fibers/Group_21/')

im_dir = '/Users/abda/Documents/Center/QIM/data/20220408_fibers/train_images_1/'
label_dir = '/Users/abda/Documents/Center/QIM/data/20220408_fibers/train_labels_1/'

if not os.path.isdir(im_dir):
    os.mkdir(im_dir[:-1])

if not os.path.isdir(label_dir):
    os.mkdir(label_dir[:-1])

it = 0
for data_dir in data_dirs:
    im_names = glob.glob(data_dir + '*.tiff')
    im_names.sort()
    for i, im_name in enumerate(im_names):
        im_new = np.array(PIL.Image.open(im_name))
        prob = model.segment_new(im_new)
        seg = insegtpy.utils.segment_probabilities(prob)
        seg[seg==0] = 1
        skimage.io.imsave(os.path.join(label_dir, f'label_{it:03d}.png'), (255*(seg-1)).astype(np.uint8))
        skimage.io.imsave(os.path.join(im_dir, f'image_{it:03d}.png'), im_new.astype(np.uint8))
        it += 1